<a href="https://colab.research.google.com/github/Akmazad/deepBrain/blob/master/Training%20on%20Google%20Colab/DL_input_Test_T_tfSpecific_layerExp4_CoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import argparse
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.nn.functional as tfunc
import torch.optim as topti
import logging
import os
import torch.cuda
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import torch.tensor

class config():
    def __init__(self):
        self.name = "deepbrainStaticConvnet_chr1_T_tfSpecific_Layer_exp4"
        self.DataDir = "gdrive/My Drive/200_400/"
        #     self.TrainingDataFile = "tempTrain5_trainingData_TF_filtered_chr5_value.npy"
        #     self.TrainingLabelFile = "tempTrain5_trainingData_TF_filtered_chr5_label_all.npy"
        #     self.TestingDataFile = "tempTrain5_validationData_TF_filtered_chr5_value.npy"
        #     self.TestingLabelFile = "tempTrain5_validationData_TF_filtered_chr5_label_all.npy"

        self.TrainingDataFile = "HumanFC_ENCODE_EpiMap_tf_specific.bin.Seq_Labels_trainingData_chr1_value.npy"
        self.TrainingLabelFile = "HumanFC_ENCODE_EpiMap_tf_specific.bin.Seq_Labels_trainingData_chr1_label_all.npy"
        self.TestingDataFile = "HumanFC_ENCODE_EpiMap_tf_specific.bin.Seq_Labels_validationData_chr1_value.npy"
        self.TestingLabelFile = "HumanFC_ENCODE_EpiMap_tf_specific.bin.Seq_Labels_validationData_chr1_label_all.npy"

        self.w_lr = 1e-2
        self.w_lr_min = 8e-7
        self.w_momentum = 0.9
        self.w_weight_decay = 5e-7
        self.l1_sparsity = 1e-20    # changed
        # self.l1_sparsity = 0
        self.print_freq = 50
        self.BATCH_SIZE = 256
        self.val_batch_size = 256
        self.seed = 0
        self.workers = 4
        self.world_size = -1
        self.rank = 0
        self.dist_url = 'env://'
        self.dist_backend = 'nccl'
        self.gpu = None
        self.multiprocessing_distributed = True
        self.nEpochs = 100
        self.start_epoch = 0
        self.resume = True

        # architecture-related parameters
        # 3 conv-layers, 1 fully connected layers (See DeepSEA paper)
        self.CONV1_INPUT_CHANNELS = 4
        self.CONV1_OUTPUT_CHANNELS = 256
        self.CONV2_OUTPUT_CHANNELS = 512
        self.CONV3_OUTPUT_CHANNELS = 1024
        self.KERNEL_SIZE = 8
        self.POOLING_TH = 4
        self.DROPOUT_block2 = 0.3
        self.DROPOUT_block3 = 0.3
        # self.DROPOUT_l3 = 0.5
        self.NUM_OUTPUTS = 128
        self.SEQ_LEN = 1000


best_acc1 = 0

# model zone
class DeeperDeepSEA(nn.Module):
    """
    A deeper DeepSEA model architecture.
    Parameters
    ----------
    sequence_length : int
        The length of the sequences on which the model trains and and makes
        predictions.
    n_targets : int
        The number of targets (classes) to predict.
    Attributes
    ----------
    conv_net : torch.nn.Sequential
        The convolutional neural network component of the model.
    classifier : torch.nn.Sequential
        The linear classifier and sigmoid transformation components of the
        model.
    """

    def __init__(self, sequence_length, n_targets, args):
        super(DeeperDeepSEA, self).__init__()
        conv_kernel_size = 8
        pool_kernel_size = 4

        self.conv_net = nn.Sequential(
            nn.Conv1d(args.CONV1_INPUT_CHANNELS, args.CONV1_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(args.CONV1_OUTPUT_CHANNELS, args.CONV1_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(
                kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.BatchNorm1d(args.CONV1_OUTPUT_CHANNELS),

            nn.Conv1d(args.CONV1_OUTPUT_CHANNELS, args.CONV2_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(args.CONV2_OUTPUT_CHANNELS, args.CONV2_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(
                kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.BatchNorm1d(args.CONV2_OUTPUT_CHANNELS),
            nn.Dropout(p=args.DROPOUT_block2),

            nn.Conv1d(args.CONV2_OUTPUT_CHANNELS, args.CONV3_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(args.CONV3_OUTPUT_CHANNELS, args.CONV3_OUTPUT_CHANNELS, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(args.CONV3_OUTPUT_CHANNELS),
            nn.Dropout(p=args.DROPOUT_block2))

        reduce_by = 2 * (conv_kernel_size - 1)
        pool_kernel_size = float(pool_kernel_size)
        self._n_channels = int(
            np.floor(
                (np.floor(
                    (sequence_length - reduce_by) / pool_kernel_size)
                 - reduce_by) / pool_kernel_size)
            - reduce_by)
        self.classifier = nn.Sequential(
            nn.Linear(args.CONV3_OUTPUT_CHANNELS * self._n_channels, n_targets),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(n_targets),
            nn.Linear(n_targets, n_targets),
            nn.Sigmoid())

    def forward(self, x, args):
        """
        Forward propagation of a batch.
        """
        out = self.conv_net(x)
        reshape_out = out.view(out.size(0), args.CONV3_OUTPUT_CHANNELS * self._n_channels)
        predict = self.classifier(reshape_out)
        return predict

class LoadDataset(tdata.Dataset):
    def __init__(self, args, dataPath, dataFile, labelFile):
        # Load data from files.
        # self.inputs = np.memmap(dataPath + dataFile, mode="r").reshape(-1, args.CONV1_INPUT_CHANNELS, args.SEQ_LEN)
        # self.labels = np.memmap(dataPath + labelFile, mode="r").reshape(-1, args.NUM_OUTPUTS)

        self.inputs = np.load(dataPath + dataFile)
        self.labels = np.load(dataPath + labelFile)[:, 438:]    # only for TF signals

        self.length = len(self.labels)

    def __getitem__(self, index):
        # Return a single input/label pair from the dataset.
        inputSample = np.array(self.inputs[index], dtype=np.float32)
        labelSample = np.array(self.labels[index], dtype=np.float32)
        sample = (inputSample, labelSample)
        return sample

    def __len__(self):
        return self.length

def getAccuracyScore(predicted, target, args, logger):
    n_digits = 3
    _predicted = torch.round(predicted * 10 ** n_digits) / (10 ** n_digits)
    __predicted = torch.round(_predicted)

    # _predicted = torch.round(predicted).detach()
    __predicted = __predicted.detach()
    _target = target.detach()

    accs = np.zeros(args.NUM_OUTPUTS, dtype=np.float)
    for i in range(args.NUM_OUTPUTS):
        try:
            acc = accuracy_score(_target.cpu().numpy()[:, i], __predicted.cpu().numpy()[:, i])
            accs[i] = acc
        except ValueError as e:
            pass
            # logger.info("NA (No positive (i.e. signal) in Test region)")

    return np.median(accs[:])


def getAUCscore(predicted, target, args, logger):
    # n_digits = 3
    # _predicted = torch.round(predicted * 10 ** n_digits) / (10 ** n_digits)
    # __predicted = torch.round(_predicted)

    __predicted = predicted.detach()
    # __predicted = __predicted.detach()
    _target = target.detach()

    aucs = np.empty(args.NUM_OUTPUTS, dtype=float)
    aucs[:] = np.nan

    for i in range(args.NUM_OUTPUTS):
        try:
            auc = roc_auc_score(_target.cpu().numpy()[:, i], __predicted.cpu().numpy()[:, i])
            aucs[i] = auc
        except ValueError as e:
            pass
            # logger.info("NA (No positive (i.e. signal) in Test region)")

    return np.nanmedian(aucs[:])

def getTPRandTNR(predicted, target, args, logger):
    n_digits = 3
    _predicted = torch.round(predicted * 10 ** n_digits) / (10 ** n_digits)
    __predicted = torch.round(_predicted)

    P = __predicted.cpu()
    T = target.cpu()

    t_ones_T_bool = torch.eq(torch.ones(T.shape[0], T.shape[1], dtype=torch.float32), T[:, :])
    t_zeros_T_bool = torch.eq(torch.zeros(T.shape[0], T.shape[1], dtype=torch.float32), T[:, :])
    t_ones_T = torch.sum(t_ones_T_bool).item()
    t_zeros_T = torch.sum(t_zeros_T_bool).item()
    tpr = torch.sum(torch.eq(T[:, :], P[:, :]) * t_ones_T_bool).item() / t_ones_T if t_ones_T != 0 else 0
    tnr = torch.sum(torch.eq(T[:, :], P[:, :]) * t_zeros_T_bool).item() / t_zeros_T if t_zeros_T != 0 else 0

    return [tpr, tnr]

def get_logger(file_path):
    """ Make python logger """
    # [!] Since tensorboardX use default logger (e.g. logging.info()), we should use custom logger
    logger = logging.getLogger('db2')
    log_format = '%(asctime)s | %(message)s'
    formatter = logging.Formatter(log_format, datefmt='%m/%d %I:%M:%S %p')
    file_handler = logging.FileHandler(file_path)
    file_handler.setFormatter(formatter)
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.setLevel(logging.INFO)

    return logger


def find_perc_uncertainty(output, low, high):
    n_digits = 3
    output = torch.round(output * 10 ** n_digits) / (10 ** n_digits)
    output = output.detach().cpu().numpy()
    return np.sum(np.logical_and(output >= low,
                                 output <= high)) / output.size  # returns the proportion of tensor elements are within the range


def train(train_loader, model, criterion, optimizer, epoch, args, logger, device):
    # switch to train mode
    model.train()
    # perc_uncertainty = 0.0

    for i, (input, target) in enumerate(train_loader):

        input = input.cuda(device, non_blocking=True)
        target = target.cuda(device, non_blocking=True)
        # print(target.shape)
        # compute output
        # output = model(input, args)
        output = model(input, args)
        loss = criterion(output, target)
        temp_target = target
        temp_pred = output
        nOnes_target = torch.sum(temp_target.detach()).item()
        temp_pred = torch.round(temp_pred * 10 ** 3) / (10 ** 3)
        temp_pred = torch.round(temp_pred)
        nOnes_predicted = torch.sum(temp_pred.detach()).item()
        tprs_tnrs = getTPRandTNR(output, target, args, logger)

        # measure accuracy and record loss
        # acc = getCustomAccuracy(output, target, args)
        perc_uncertainty = find_perc_uncertainty(output, 0.4, 0.6)
        # custom_accuracy = getCustomAccuracy2(output, target, args)
        aucs = getAUCscore(output, target, args, logger)
        accs = getAccuracyScore(output, target, args, logger)
        # f1s = getF1Score(output, target, args, logger)

        # compute gradient
        optimizer.zero_grad()

        # add l1 Sparsity
        l1 = 0
        for p in model.parameters():
            l1 = l1 + p.abs().sum()
        loss = loss + args.l1_sparsity * l1

        # do SGD step
        loss.backward()
        optimizer.step()

        if i % args.print_freq == 0 or i == len(train_loader) - 1:
            #           logger.info("TRAINING: Epoch: %d, Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, custom[HumanFC:%.3f, EpiMap:%.3f, TFs:%.3f], roc[HumanFC:%.3f, EpiMap:%.3f, TFs:%.3f]"
            #                       % (epoch + 1, i, len(train_loader) - 1,
            #                          loss, perc_uncertainty, custom_accuracy[0],
            #                          custom_accuracy[1], custom_accuracy[2],
            #                          aucs[0], aucs[1], aucs[2]))
            logger.info(
                # "TRAINING: Epoch: %d, Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, custom[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f], roc[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ], accuracy[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ], f1[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ]"
                "TRAINING: Epoch: %d, Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, nOnes_target: %d, "
                "nOnes_pred: %d, TPR[ TFs: %.3f], TNR[ TFs: %.3f], roc[ TFs: %.3f ], accuracy[ TFs: %.3f ]"
                % (epoch + 1, i, len(train_loader) - 1,
                   loss, perc_uncertainty, nOnes_target, nOnes_predicted,
                   tprs_tnrs[0],
                   tprs_tnrs[1],
                   aucs,
                   accs))


def validate(val_loader, model, criterion, args, logger, device, epoch):
    # switch to evaluate mode
    model.eval()
    total_ACC, total_RNA, total_TFs = 0, 0, 0

    # losses = []

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):

            input = input.cuda(device, non_blocking=True)
            target = target.cuda(device, non_blocking=True)

            # print(target.shape)

            output = model(input, args)
            loss = criterion(output, target)
            temp_target = target
            temp_pred = output
            nOnes_target = torch.sum(temp_target.detach()).item()
            temp_pred = torch.round(temp_pred * 10 ** 3) / (10 ** 3)
            temp_pred = torch.round(temp_pred)
            nOnes_predicted = torch.sum(temp_pred.detach()).item()
            tprs_tnrs = getTPRandTNR(output, target, args, logger)

            # losses[i] = loss

            # measure accuracy and record loss
            # acc = getCustomAccuracy(output, target, args)
            p = find_perc_uncertainty(output, 0.4, 0.6)
            # custom_accuracy = getCustomAccuracy2(output, target, args)
            aucs = getAUCscore(output, target, args, logger)
            accs = getAccuracyScore(output, target, args, logger)
            # f1s = getF1Score(output, target, args, logger)



            if i % args.print_freq == 0 or i == len(val_loader) - 1:
                # progress._print(i)
                # logger.info("batch: %d, loss: %.3f; valid accuracy: custom_accuracy_metric: %.3f, ACC marks: %.3f, RNA-seq: %.3f, TFs: %.3f" % (i+1, loss, acc, aucs[0], aucs[1], aucs[2]))
                #                 logger.info("VALIDATION: Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, custom[HumanFC:%.3f, EpiMap:%.3f, TFs:%.3f], roc[HumanFC: %.3f, EpiMap: %.3f, TFs:%.3f]" % (i, len(val_loader)-1, loss, perc_uncertainty, custom_accuracy[0], custom_accuracy[1], custom_accuracy[2], aucs[0], aucs[1], aucs[2]))
                logger.info(
                    "VALIDATION: Epoch: %d, Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, nOnes_target: %d, "
                    "nOnes_pred: %d, TPR[ TFs: %.3f], TNR[ TFs: %.3f], roc[ TFs: %.3f ], accuracy[ TFs: %.3f ]"
                    # "VALIDATION: Batch: %d/%d, Loss: %.3f, perc_uncertainty: %.3f, custom[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f], roc[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ], accuracy[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ], f1[HumanFC: %.3f, EpiMap: %.3f, TFs: %.3f ]"
                    % (epoch + 1, i, len(val_loader) - 1,
                       loss, p, nOnes_target, nOnes_predicted,
                       tprs_tnrs[0],
                       tprs_tnrs[1],
                       aucs,
                       accs))




    # return np.median(losses), np.median([total_ACC, total_RNA, total_TFs])


def adjust_learning_rate(optimizer, epoch, args, lr_scheduler):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    # lr = args.w_lr * (0.1 ** (epoch // 30))   # option 1
    lr_scheduler.step()  # option 2
    lr = lr_scheduler.get_lr()[0]

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def load_ckp(checkpoint_fpath, model, optimiser):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimiser.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimiser, checkpoint['epoch']

def main():
    # Training settings
    args = config()
    use_cuda = torch.cuda.is_available()
    torch.manual_seed(args.seed)
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)

    DataPath = args.DataDir
    if not (os.path.exists(os.path.join(DataPath, args.name))):
        os.mkdir(os.path.join(DataPath, args.name))
    f = open(os.path.join(DataPath, args.name) + "/" + "{}.log".format(args.name), "a")
    f.close()
    logger = get_logger(os.path.join(os.path.join(DataPath, args.name), "{}.log".format(args.name)))

    trainDataset = LoadDataset(args, dataPath=DataPath, dataFile=args.TrainingDataFile,
                               labelFile=args.TrainingLabelFile)
    # define sampler
    train_sampler = torch.utils.data.sampler.RandomSampler(trainDataset)

    trainLoader = tdata.DataLoader(trainDataset, batch_size=args.BATCH_SIZE, shuffle=(train_sampler is None),
                                   num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    # Load the testing dataset, and create a data loader to generate a batch. CHANGE THIS ONCE TESTING DATASET IS READY
    valDataset = LoadDataset(args, dataPath=DataPath, dataFile=args.TestingDataFile, labelFile=args.TestingLabelFile)
    valLoader = tdata.DataLoader(dataset=valDataset, batch_size=args.val_batch_size, shuffle=False,
                                 num_workers=args.workers, pin_memory=True)

    # build the model and criterion
    # model = BuildModel(args).to(device)
    # model = DeepSEA(args.SEQ_LEN, args.NUM_OUTPUTS).to(device)
    model = DeeperDeepSEA(args.SEQ_LEN, args.NUM_OUTPUTS, args).to(device)
    # model = get_model(load_weights=False)

    # Add a sigmoid activation function to the output.  Use a binary cross entropy
    criterion = nn.BCELoss().to(device)
    # criterion = nn.CrossEntropyLoss()

    # optimiser = topti.Adam(model.parameters(), lr=args.w_lr)  # Minimise the loss using the Adam algorithm.
    #     optimiser = torch.optim.Adam(model.parameters(), args.w_lr, betas=(0.5, 0.999),
    #                                    weight_decay=args.w_weight_decay)
    # print(model.parameters())
    optimiser = torch.optim.SGD(model.parameters(), args.w_lr, momentum=args.w_momentum,
                                weight_decay=args.w_weight_decay)

    # model = DeepSEA(args.SEQ_LEN, args.NUM_OUTPUTS).to(device)
    # criterion = nn.BCELoss()
    # optimiser = (torch.optim.SGD, {"lr": args.w_lr, "weight_decay": 1e-6, "momentum": 0.9})

    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, args.nEpochs, eta_min=args.w_lr_min)
    # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, args.nEpochs, eta_min=0)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min')


    if args.resume:
      ckp_path = os.path.join(DataPath, args.name) + "/" + "deepBrain_TF_model_LayerExp4.pt"
      model, optimiser, args.start_epoch = load_ckp(ckp_path, model, optimiser)
      model.train()

    best_acc1 = 0.0
    for epoch in range(args.start_epoch, args.start_epoch + args.nEpochs):

        # train for one epoch
        train(trainLoader, model, criterion, optimiser, epoch, args, logger, device)

        # adjust learning rate
        adjust_learning_rate(optimiser, epoch, args, lr_scheduler)

        # evaluate on validation set
        validate(valLoader, model, criterion, args, logger, device, epoch)
        
        # torch.save(model.state_dict(), "deepBrain_TF_model_LayerExp4.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimiser.state_dict()
            }, os.path.join(DataPath, args.name) + "/" + "deepBrain_TF_model_LayerExp4.pt")

    # if (args.save_model):
    #     torch.save(model.state_dict(), "mnist_cnn.pt")
    # torch.save(model.state_dict(), "deepBrain2_cnn.pt")


if __name__ == '__main__':
    main()

cuda


12/04 11:28:47 PM | TRAINING: Epoch: 48, Batch: 0/8663, Loss: 0.095, perc_uncertainty: 0.019, nOnes_target: 2382, nOnes_pred: 1837, TPR[ TFs: 0.627], TNR[ TFs: 0.989], roc[ TFs: 0.977 ], accuracy[ TFs: 0.969 ]
12/04 11:29:14 PM | TRAINING: Epoch: 48, Batch: 50/8663, Loss: 0.082, perc_uncertainty: 0.017, nOnes_target: 1973, nOnes_pred: 1609, TPR[ TFs: 0.633], TNR[ TFs: 0.988], roc[ TFs: 0.980 ], accuracy[ TFs: 0.973 ]
12/04 11:29:41 PM | TRAINING: Epoch: 48, Batch: 100/8663, Loss: 0.096, perc_uncertainty: 0.020, nOnes_target: 2358, nOnes_pred: 1774, TPR[ TFs: 0.606], TNR[ TFs: 0.989], roc[ TFs: 0.976 ], accuracy[ TFs: 0.967 ]
12/04 11:30:07 PM | TRAINING: Epoch: 48, Batch: 150/8663, Loss: 0.094, perc_uncertainty: 0.019, nOnes_target: 2561, nOnes_pred: 1975, TPR[ TFs: 0.648], TNR[ TFs: 0.990], roc[ TFs: 0.976 ], accuracy[ TFs: 0.965 ]
12/04 11:30:34 PM | TRAINING: Epoch: 48, Batch: 200/8663, Loss: 0.089, perc_uncertainty: 0.017, nOnes_target: 2217, nOnes_pred: 1832, TPR[ TFs: 0.661], TNR